# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [3]:
import json
import time
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm
import string
from importlib import reload

# to help generate formatted output rnepresentations
from IPython.core.display import display, HTML
import markdown2 as md

def html(markdown_string):
    display(HTML(md.markdown(markdown_string)))
    
# for simplifying text to just numbers
no_punctuation = str.maketrans(string.punctuation+string.ascii_letters, ' '*len(string.punctuation+string.ascii_letters))

# identify game archive and votecounter of interest
archive = '../data/archive.txt'
votecounter = VoteCounter

## Helper Function(s)

In [4]:
def _relevantGameInfo(game):
    
    # link
    link = game[:game.find('\n')]
    
    # thread number
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])

    # special events
    lessOneForMislynch, events, notes = False, {}, game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):]
    for note in notes.split('; '):
        note = note.replace(' in post ', ' on post ')
        if ' on post ' in note:
            postnumber = note.split(' on post ')[1].replace(';', '')
            postnumber = postnumber[:postnumber.find(' but')] if 'but' in postnumber else postnumber
            if postnumber in events:
                events[postnumber].append(note.split(' on post ')[0])
            else:
                events[postnumber] = [note.split(' on post ')[0]]
        elif 'one less for no lynch' in note.lower():
            lessOneForMislynch = True
    
    # game title and number
    title = game.split('\n')[1]
    title_number = [i for i in title.translate(no_punctuation).split() if i.isdigit()][0]
    
    # moderator(s)
    moderators = game.split('\n')[2][len('Moderator: '):].split(', ')
    
    # living slots/players for each Day
    doublevoters = []
    slots, players, fates, lynched = [], [], [], {}
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        
        # build list of players and slots
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        
        # extract role and check for doublevoter
        if 'double voter' in line[1].lower() or 'doublevoter' in line[1].lower():
            doublevoters.append(line[0].split(' replaced '))
        
        # extract last phase slot's vote helped decide Day
        # it's the phase they died, minus one if they were day-killed (not lynched)
        if 'survived' in line[-1].lower() or 'endgamed' in line[-1].lower():
            fates.append(float('inf'))
        else:
            phase = int(line[-1][line[-1].rfind(' ')+1:])
            #fate_modifier = 'killed day' in line[-1][:line[-1].rfind(' ')].lower() 
            fates.append(max(0, phase))
        
        # sort any detected lynches into `lynched` array
        if 'lynched' in line[-1].lower():
            lynched[phase] = slots[-1]

    return slots, players, fates, lynched, number, transitions[title_number], moderators, events, doublevoters, lessOneForMislynch

## VotecounterTest

In [5]:
print('going...')

#parameters
start_index = 10
end_index = 0

reload(VoteCounter)

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# open transitions archive, convert into dictionary of lists
with open('../data/transitions.tsv') as f:
    transitions = f.read()
transitions = transitions.split('\n')[3:]
transitions = {line.split('\t')[0]:line.strip().split('\t')[1:] for line in transitions if len(line.strip().split('\t')[1:]) > 0}

# process votes in each game's posts until a lynch found
# then store information about votecounter's performance
vote_results, vote_success, transition_results, transition_success, t0, total = {}, 0, {}, 0, time.time(), 0
end_index = end_index if end_index else len(games)  
for game_index, game in enumerate(games[start_index:end_index]):
    
    # temporarily only consider games where... (MODIFIABLE)
    if False:#not any(each in game for each in ['Game 1501']):
        continue
    
    # extract relevant information about this game
    slots, players, fates, lynched, number, game_transitions, moderators, events, doublevoters, lessOneForMislynch = _relevantGameInfo(game)
    #events = {} ### REMOVE THIS WHEN NOT TESTING FOR ERRORS
    with open('../data/posts/{}.jsonl'.format(number)) as f:
        gameposts =  [json.loads(l) for l in f]
        
    # prepare to collect data for this game
    transition_results[number] = []
    vote_results[number] = []
    
    for day in range(1, len(game_transitions)):
        
        # we'll remove this later; don't scan games that don't have this many games
        if len(game_transitions) < day+1:
            continue
    
        # identify day-specific information and set up voteextractor and votecount for them
        canPredictTransition, canPredictLynch = True, True
        if f'd{day} long twilight' in game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):].lower():
            canPredictTransition = False
        if f'd{day} hammer after deadline' in game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):].lower():
            canPredictLynch = False
        if f'd{day} no majority' in game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):].lower():
            correct = None
            canPredictTransition = False
        elif f'd{day} no lynch' in game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):].lower():
            correct = 'NO LYNCH'
        else:
            correct = lynched[day] if day in lynched else None
            
        start_point = 0 if day == 1 else int(game_transitions[day-2])
        end_point = int(game_transitions[day-1])+1 if not correct else len(gameposts)
        relevant_slots = [slot for slot_index, slot in enumerate(slots) if fates[slot_index] >= day]
        relevant_players = []
        for slot in relevant_slots:
            relevant_players += slot
        votecount = VoteCount(relevant_slots, meta={'correct': correct}, lessOneForMislynch=lessOneForMislynch, doublevoters=doublevoters)
        votecounter = VoteCounter.VoteExtracter(players=relevant_players)

        tphase, transition_start, transition_end, transition_match, transition_url = time.time(), None, None, False, None
        for post in gameposts[start_point:end_point]:
            
            # prioritize any events associated with post
            if post['number'] in events:
                post_events = events[post['number']]
                for event in post_events:

                    # if event is a daykill, remove the player from votecount and voteextractor
                    if 'killed' == event.split(' ')[-1]:
                        
                        # update relevant slots and players and make new votecounter
                        killed_player = event[:event.rfind(' ')]
                        killed_slot = next(s for s in relevant_slots if s.count(killed_player) > 0)
                        del relevant_slots[relevant_slots.index(killed_slot)]
                        relevant_players = []
                        for slot in relevant_slots:
                            relevant_players += slot
                        votecounter = VoteCounter.VoteExtracter(players=relevant_players)
                        votecount.killplayer(killed_player, post['number'])
                        
                    # if event is a vote reset, set relevant player(s) to not voting
                    elif 'reset' == event.split(' ')[1]:
                        reset_players = ([s[0] for s in relevant_slots] 
                                        if event.split(' ')[0].lower() == 'votecount'
                                         else [event.split(' ')[0]])
                        for reset_player in reset_players:
                            votecount.update(reset_player, 'UNVOTE', post['number'])
                            
                    # if event is a vote specification, set relevant player(s) to vote
                    elif ' voted ' in event:
                        votecount.update(event.split(' voted ')[0], event.split(' voted ')[1], post['number'])

            # consider no more votes if voters have made a choice already
            elif not votecount.choice:

                # ignore posts not made by players
                if relevant_players.count(post['user']) == 0:
                    continue

                # update votecount for each vote found by votecounter
                # stop considering votes in post if votecount.choice
                for voted in votecounter.fromPost(post):
                    votecount.update(post['user'], voted, post['number'])
                    if votecount.choice:
                        break

            # keep scanning to find newest post by game mod after detectedhammer
            elif not transition_start:
                if moderators.count(post['user']) > 0:
                    transition_start = int(post['number'])
                    transition_url = post['pagelink']

            # keep scanning to find last successive post by mod after they end Day
            elif not transition_end:
                if moderators.count(post['user']) == 0:
                    transition_end = int(post['number'])

                    # track match between inferred and transcribed transition post#
                    transition_match = int(game_transitions[day-1]) in list(range(transition_start, transition_end))

            # finish if votecount.choice, transition_start, and transition_end all populated
            else:
                break

        vote_success += votecount.choice == correct
        transition_success += transition_match
        transition_results[number].append([list(range(transition_start, transition_end)), transition_url] if transition_start and transition_end else "None")
        vote_results[number].append(votecount)
        total += 1
        #if not (((not canPredictLynch) or votecount.choice == correct) and ((not canPredictTransition) or transition_match)):
        print(day)
        print(game)
        #print(game.split('\n\n')[0])
        print(f'{game_index + start_index} {number} {vote_success} {transition_success} {total} {votecount.choice == correct} {transition_match} {time.time()-tphase}')
        print('\n---\n')

print(f'{vote_success/total} {transition_success/total} {total} {time.time()-t0}')

going...


IndexError: list index out of range

## Analyze Results
Will have to revise this to adapt to new variable names/codeflow.

In [ ]:
# parameters
game_index = 6
day = 1
postnumber = 0
postnumber = str(postnumber)


reload(VoteCounter)

print(game_index)
print(day)

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# generate votecounter w/ specified players
game = games[game_index]
slots, players, fates, lynched, number, game_transitions, moderators, events, doublevoters, lessOneForMislynch = _relevantGameInfo(game)
events = {} ### REMOVE THIS WHEN NOT TESTING FOR ERRORS

print(events)

# temporary DAY 2fix for until i run every day at once
#if vote_results[number][0]:
#    vote_results[number] = [None, None, vote_results[number][0]]
#    transition_results[number] = [None, None, transition_results[number][0]]

if f'd{day} no majority' in game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):].lower():
    correct = None
elif f'd{day} no lynch' in game[:game.find('\n\n')].split('\n')[-1][len("Notes: "):].lower():
    correct = 'NO LYNCH'
else:
    correct = lynched[day] if day in lynched else None
start_point = 0 if day == 1 else int(game_transitions[day-2])
end_point = int(game_transitions[day-1])
if not int(postnumber):
    postnumber = game_transitions[day-1]
    
html('# {}'.format(number))
print(games[game_index])
print()
print('Vote Choice:', vote_results[number][day-1].choice)
print('Vote Correct:', vote_results[number][day-1].meta['correct'])

# relevant information and votecounter generation
relevant_slots = [slot for slot_index, slot in enumerate(slots) if fates[slot_index] >= day]
relevant_players = []
for slot in relevant_slots:
    relevant_players += slot
votecount = VoteCount(relevant_slots, meta={'correct': correct}, lessOneForMislynch=lessOneForMislynch, doublevoters=doublevoters)
votecounter = VoteCounter.VoteExtracter(players=relevant_players)

# collect gameposts associated with game number
with open('../data/posts/{}.jsonl'.format(number)) as f:
    gameposts = [json.loads(l) for l in f]
    
# find and display selected post along with votecounter output for it
html('# Post {}'.format(postnumber))
post = next(item for item in gameposts if item["number"] == postnumber)
print('Extracted Votes:', list(votecounter.fromPost(post)))
print()
print(post)
print()
display(HTML(post['content']))
    
# display extracted and true phase transitions
html('# Phase Transitions')
print('True Transitions:', game_transitions)
print(transition_results[number][day-1])
html('[{}]({})'.format(*transition_results[number][day-1]))
transition_start, transition_end, transition_match, final_vote = None, None, False, None
for post in gameposts[start_point:int(postnumber)]:
    
    # prioritize any events associated with post
    if post['number'] in events:
        post_events = events[post['number']]
        for event in post_events:
            
            # if event is a daykill, remove the player from votecount and voteextractor
            if 'killed' == event.split(' ')[-1]:

                # update relevant slots and players and make new votecounter
                killed_player = event[:event.rfind(' ')]
                print(killed_player)
                print('------------------------------------------')
                killed_slot = next(s for s in relevant_slots if s.count(killed_player) > 0)
                del relevant_slots[relevant_slots.index(killed_slot)]
                relevant_players = []
                for slot in relevant_slots:
                    relevant_players += slot
                votecounter = VoteCounter.VoteExtracter(players=relevant_players)
                votecount.killplayer(killed_player, post['number'])
                
            # if event is a vote reset, set relevant player(s) to not voting
            elif 'reset' == event.split(' ')[1]:
                reset_players = ([s[0] for s in relevant_slots] 
                                if event.split(' ')[0].lower() == 'votecount'
                                 else [event.split(' ')[0]])
                for reset_player in reset_players:
                    votecount.update(reset_player, 'UNVOTE', post['number'])
                    
            # if event is a vote specification, set relevant player(s) to vote
            elif ' voted ' in event:
                votecount.update(event.split(' voted ')[0], event.split(' voted ')[1], post['number'])
                        
    # consider no more votes if voters have made a choice already
    elif not votecount.choice:
        # ignore posts not made by players
        if relevant_players.count(post['user']) == 0:
            continue

        # update votecount for each vote found by votecounter
        # stop considering votes in post if votecount.choice
        for voted in votecounter.fromPost(post):
            votecount.update(post['user'], voted, post['number'])
            if votecount.choice:
                print('Final Vote Post#', post['number'])
                break

    # keep scanning to find newest post by game mod after detectedhammer
    elif not transition_start:
        if moderators.count(post['user']) > 0:
            transition_start = int(post['number'])
            print('Transition Start:', transition_start)

    # keep scanning to find last successive post by mod after they end Day
    elif not transition_end:
        if moderators.count(post['user']) == 0:
            transition_end = int(post['number'])
            print('Transition End:', transition_start)

            # track match between inferred and transcribed transition post#
            transition_match = int(game_transitions[0]) in list(range(transition_start, transition_end))
            print('Transition Match:', transition_match)

    # finish if votecount.choice, transition_start, and transition_end all populated
    else:
        break
        
# display votecount up to indicated post number
html('# Current Votecount (Up to {})'.format(postnumber))
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

# display final votecount
html('# Final Votecount')
final_votecount = vote_results[number][day-1].todict()
for each in final_votecount:
    if final_votecount[each]:
        print(each, '-', len(final_votecount[each]))
        for voter in final_votecount[each]:
            print(voter)
        print()
        
# display final votelog
html('# Vote Log')
votelog = vote_results[number][day-1].votelog.copy()
for index, each in enumerate(votelog):
    each = each.split()
    each[-1] = '[{}]({})'.format(each[-1], next(item for item in gameposts if item["number"] == each[-1])['pagelink'])
    votelog[index] = ' '.join(each)
html('  \n'.join(reversed(votelog)))

In [ ]:
post = gameposts[1219]

import os

# to detect slight misspellings
import editdistance as ed
    
# spellchecker to help identify english words
from spellchecker import SpellChecker
spell = SpellChecker()

# to help parse website content
from lxml import html
    
# sadly we rely on regular expressions
import re

# regex filters
regall = re.compile('[^a-zA-Z]') # any character that IS NOT a-z OR A-Z
regup = re.compile('[^A-Z]') # any character that IS NOT A-Z

# paths useful for finding votes in posts
votepath1 = '/html/body/p/span[@class="{}"]//text()'
votepath2 = '/html/body/span[@class="{}"]//text()'
votepath3 = '/html/body/p/span/span[@class="{}"]//text()'
votepath4 = '/html/body/span/span[@class="{}"]//text()'
votepath5 = '/html/body/p/span[@class="{}"]'
votepath6 = '/html/body/span[@class="{}"]'
subpath = 'span//text()'

sel = html.fromstring('<html><body>' + post['content'] + '</body></html>')

# pull out all relevant tags
tagclass = 'noboldsig'
boldtags = (sel.xpath(votepath1.format(tagclass)) +
            sel.xpath(votepath2.format(tagclass)) +
            sel.xpath(votepath3.format(tagclass)) +
            sel.xpath(votepath4.format(tagclass)) +
            [''.join(each.xpath(subpath)) for each 
             in sel.xpath(votepath5.format(tagclass))] +
            [''.join(each.xpath(subpath)) for each 
             in sel.xpath(votepath6.format(tagclass))]
           )
tagclass = 'bbvote'
votetags = (sel.xpath(votepath1.format(tagclass)) +
            sel.xpath(votepath2.format(tagclass)) +
            sel.xpath(votepath3.format(tagclass)) +
            sel.xpath(votepath4.format(tagclass)) +
            [''.join(each.xpath(subpath)) for each 
             in sel.xpath(votepath5.format(tagclass))] +
            [''.join(each.xpath(subpath)) for each 
             in sel.xpath(votepath6.format(tagclass))]
           )

# first of all, though, we handle broken bold tags similarly
# after some preprocessing, so let's add those
for content in (sel.xpath('/html/body/text()') +
                sel.xpath('/html/body/p/text()')):
    if content.count('[/b]') > 0:
        # up to broken tag
        tagline = content[:content.find('[/b]')].lstrip().rstrip()
        boldtags.append(tagline)
    if content.count('[b]') > 0:
        # starting at broken tag
        tagline = content[content.find('[b]')+3:].lstrip().rstrip()
        boldtags.append(tagline)

#  we want votetags to have priority, so add them to the pool here
boldtags = boldtags + votetags
boldtags = [b.rstrip().lstrip() for b in boldtags]

# they need to have 'vote' or 'veot' early in their string
boldtags = [b for b in boldtags
            if b[:7].lower().count('vote') or b[:7].lower().count('veot') > 0 or b[:7].lower().count('vtoe') > 0 or b[:7].lower().count('ovte') > 0]

# rfind 'vote' and 'unvote' (and their key mispellings) to locate vote
for i, v in enumerate(boldtags):
    voteloc = max(v.lower().rfind('vote'), v.lower().rfind('veot'), v.lower().rfind('vtoe'), v.lower().rfind('ovte'))
    unvoteloc = max(v.lower().rfind('unvote'), v.lower().rfind('unveot'), v.lower().rfind('unvtoe'), v.lower().rfind('unovte'))

    # if position of unvote is position of vote - 2, 
    # then the last vote is an unvote
    if unvoteloc > -1 and unvoteloc == voteloc - 2:
        boldtags[i] = 'UNVOTE'

    # otherwise vote is immediately after 'vote' text and perhaps some crap
    else:
        boldtags[i] = v[voteloc+4:].replace(
            ': ', ' ').replace(':', ' ').replace('\n', ' ').rstrip().lstrip()

votes = boldtags
[v for v in votes if len(v.strip()) > 0]

In [ ]:
gameposts[16]

In [ ]:
html('# Current Votecount (Up to {})'.format(postnumber))
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()